In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain

In [0]:
dbutils.library.restartPython()

### Multi-Agents

In [0]:
import os
import sys
import asyncio
import nest_asyncio
from typing import Optional, List

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor


from databricks_langchain import ChatDatabricks 
from langgraph.graph import StateGraph, END  # pip install langgraph


In [0]:
# -----------------------
# 1) Environment / LLM
# -----------------------
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-1-8b-instruct" # Model Serving endpoint name; other option see "Serving" under AI/ML tab (e.g. databricks-gpt-oss-20b)

llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.2)

In [0]:
from typing import TypedDict, Optional

# -----------------------
# 2) State definition
# -----------------------
class BlogState(TypedDict, total=False):
    topic: str
    research: str
    blog_post: str

In [0]:
# -----------------------
# 3) Node: Researcher
# -----------------------
def researcher_node(state: BlogState) -> BlogState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a Senior Research Analyst.
Research the top 3 emerging trends in AI in 2024–2025.
Focus on practical applications and potential impact.
Include sources (links or clear citations) for each trend."""),
        ("user", "Topic: {topic}")
    ])

    chain = prompt | llm | StrOutputParser()
    research = chain.invoke({"topic": state["topic"]})
    return {"research": research}

In [0]:
# -----------------------
# 4) Node: Writer
# -----------------------
def writer_node(state: BlogState) -> BlogState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a Technical Content Writer.
Write an engaging ~500-word blog post for a general audience.
Use the research notes below. Keep it clear and practical.

Research notes:
{research}
"""),
        ("user", "Write the blog post about: {topic}")
    ])

    chain = prompt | llm | StrOutputParser()
    blog_post = chain.invoke({"topic": state["topic"], "research": state["research"]})
    return {"blog_post": blog_post}

In [0]:
# -----------------------
# 5) Build the LangGraph
# -----------------------
def build_graph():
    graph = StateGraph(BlogState)

    graph.add_node("researcher", researcher_node)
    graph.add_node("writer", writer_node)

    graph.set_entry_point("researcher")
    graph.add_edge("researcher", "writer")
    graph.add_edge("writer", END)

    return graph.compile()

In [0]:
app = build_graph()

initial_state: BlogState = {"topic": "Top 3 emerging trends in Artificial Intelligence in 2024–2025"}
final_state = app.invoke(initial_state)

print("\n================ RESEARCH ================\n")
print(final_state["research"])
print("\n================ BLOG POST ================\n")
print(final_state["blog_post"])

### Memory Manangement

In [0]:
from langgraph.graph import StateGraph, END
from langgraph.store.memory import InMemoryStore

# Dummy embedder for InMemoryStore (replace with a real embedding model in prod)
def embed(texts: list[str]) -> list[list[float]]:
    return [[1.0, 2.0] for _ in texts]

store = InMemoryStore(index={"embed": embed, "dims": 2})


In [0]:
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# -------------------------
# 1) Graph state
# -------------------------
class AgentState(TypedDict, total=False):
    user_id: str
    app: str
    messages: List[BaseMessage]          # short-term chat history
    user_input: str                      # current user turn
    instructions: str                    # current system instructions
    response: str                        # model response


In [0]:
# -------------------------
# 2) Helpers: store keys
# -------------------------
def ns(state: AgentState):
    return (state["user_id"], state["app"])

INSTR_KEY = "agent_instructions"


In [0]:
# -------------------------
# 3) Node A: load instructions (from store)
# -------------------------
def load_instructions(state: AgentState) -> AgentState:
    namespace = ns(state)
    got = store.get(namespace, INSTR_KEY)
    if got:
        instructions = got.value["instructions"]
    else:
        instructions = "You are a helpful assistant. Be concise."
        store.put(namespace, INSTR_KEY, {"instructions": instructions})
    return {"instructions": instructions}

In [0]:
# -------------------------
# 4) Node B: update instructions (reflect on convo)
# -------------------------
def update_instructions(state: AgentState) -> AgentState:
    # Reflect on the conversation and improve the system instructions
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You improve an assistant’s system instructions over time.
Given the current instructions and recent conversation, produce an updated instruction set.
Rules:
- Keep it short (<= 8 bullet points).
- Only add stable preferences or rules inferred from the conversation.
- Don’t invent personal facts.
Return only the updated instructions text."""),
        ("human", "Current instructions:\n{instructions}\n\nConversation:\n{conversation}")
    ])

    conversation_text = "\n".join(
        [f"Human: {m.content}" if isinstance(m, HumanMessage) else f"AI: {m.content}" for m in state.get("messages", [])]
    )

    chain = prompt | llm | StrOutputParser()
    new_instructions = chain.invoke({
        "instructions": state["instructions"],
        "conversation": conversation_text
    })

    store.put(ns(state), INSTR_KEY, {"instructions": new_instructions})
    return {"instructions": new_instructions}

In [0]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# -------------------------
# 5) Node C: call model (use instructions + chat history)
# -------------------------
def call_model(state: AgentState) -> AgentState:

    prompt = ChatPromptTemplate.from_messages([
        ("system", "{instructions}"),
        MessagesPlaceholder("messages"),
        ("human", "{user_input}")
    ])

    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({
        "instructions": state["instructions"],
        "messages": state.get("messages", []),
        "user_input": state["user_input"],
    })

    # append to short-term history
    new_messages = list(state.get("messages", [])) + [HumanMessage(content=state["user_input"]), AIMessage(content=response)]
    return {"response": response, "messages": new_messages}

In [0]:
# -------------------------
# 6) Build graph: load -> call -> update -> end
# -------------------------
def build_graph():
    g = StateGraph(AgentState)
    g.add_node("load_instructions", load_instructions)
    g.add_node("call_model", call_model)
    g.add_node("update_instructions", update_instructions)

    g.set_entry_point("load_instructions")
    g.add_edge("load_instructions", "call_model")
    g.add_edge("call_model", "update_instructions")
    g.add_edge("update_instructions", END)

    return g.compile()


In [0]:
app = build_graph()
state: AgentState = {
    "user_id": "my-user",
    "app": "chitchat",
    "messages": [],
    "user_input": "Hi, I'm Ling. Please keep answers short.",
}
out = app.invoke(state)
print("Assistant:", out["response"])

state = {
    "user_id": "my-user",
    "app": "chitchat",
    "messages": out["messages"],
    "user_input": "Do you remember my preference?",
}
out2 = app.invoke(state)
print("Assistant:", out2["response"])